<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/04_%EB%A6%AC%EB%B7%B0%EB%B6%84%EC%84%9D_%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os
from tqdm.notebook import tqdm

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# DATA_PATH = "/content/drive/MyDrive/프로젝트/final project/data/"
DATA_PATH = "/content/drive/MyDrive/project_final/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.4 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.2 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# 모델과 토크나이저 초기화
model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
model = AutoModelForSequenceClassification.from_pretrained(model_name) #.to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# 전처리된 데이터 불러오기

- 서비스1(답변생성 서비스)에서 전처리가 완료된 데이터

In [ ]:
train_ft = pd.read_csv(f"{DATA_PATH}train_0921.csv")
rename_cols = ['review','reply','sentiment','food','store','branch']
train_ft.columns = rename_cols
train_ft = train_ft[['review','reply','sentiment']]
train_ft

,review,reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0
...,...,...,...
23757,맛있어요.! 된장찌개에서 멜젓 맛도 나고 맛이있네용 !다만 커피는 제입맛엔 살짝 연...,저희 삼겹살에 찌개까지 맛있게 잘 드셨다니 기쁘고 보람됩니다. 이렇게 소중한 리뷰와...,1
23758,애들이 이것만 찾아요 바삭하고 짭쪼름하고 맛있어요,맛있게 드셨다니 감사합니다 항상 노력하는 7번가 피자 용암점 되겠습니다 즐거운 시간...,1
23759,탕수육 맛있어요 진짜 맛있어요 뭔가 고기가 꽉찬 느낌? 부먹 찍먹 구분없이 그냥 탕...,안녕하세요 너무 감사드립니다 새해에도 건강하시고 행복하세요 감사드립니다 ~,1
23760,면이 조금 불었어 아쉽지만 그래도 맛있게 잘 먹었습니다.,안녕하세요 고객님 소중한 리뷰 남겨주셔서 감사합니다. 맛있게 드셨다고 하시니 너무 ...,1


## 추가 전처리

- 데이터 정제

In [ ]:
review_list = []
for review in train_ft['review']:
    if isinstance(review, str):

        pat = re.compile("[^가-힣0-9.,~?!]") # 영어 제거
        result = pat.sub(" ", review)

        result = re.sub(r'^[.]\s*', '', result, count=1)
        result = re.sub(r'^[. ]+', '', result, count=1)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train_ft['review'] = review_list
train_ft.head()

,review,reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0


In [ ]:
reply_list = []
for reply in train_ft['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^가-힣0-9.,~?!]") # 영어 제거
        result = pat.sub(" ", reply)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^[.]\s*', '', result, count=1)
        result = re.sub(r'^[. ]+', '', result, count=1)

        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train_ft['reply'] = reply_list
train_ft.head()

,review,reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0


- 한글이 포함되지 않은 리뷰 제거

In [ ]:
pattern = r'^[^가-힣]*$'
# 패턴을 포함하는 문자열을 찾아 mask 생성
mask = (train_ft['review'].str.match(pattern)) | (train_ft['reply'].str.match(pattern))

train_ft = train_ft[~mask]

- 고객 id처리

In [ ]:
# 님 앞의 문자들 고객님으로 처리. 짧은 리뷰 제거 이전 처리함.
train_ft['reply'] = train_ft['reply'].str.replace(r'.*님', '고객님', regex=True)

<ipython-input-13-6c345ec58d5b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ft['reply'] = train_ft['reply'].str.replace(r'.*님', '고객님', regex=True)


- 짧은 리뷰 제거

In [ ]:
mask = train_ft['reply'].str.len() <= 10
train_ft[mask]

train_ft = train_ft[~mask]
train_ft

,review,reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0
...,...,...,...
23757,맛있어요.! 된장찌개에서 멜젓 맛도 나고 맛이있네용 !다만 커피는 제입맛엔 살짝 연...,저희 삼겹살에 찌개까지 맛있게 잘 드셨다니 기쁘고 보람됩니다. 이렇게 소중한 리뷰와...,1
23758,애들이 이것만 찾아요 바삭하고 짭쪼름하고 맛있어요,맛있게 드셨다니 감사합니다 항상 노력하는 7번가 피자 용암점 되겠습니다 즐거운 시간...,1
23759,탕수육 맛있어요 진짜 맛있어요 뭔가 고기가 꽉찬 느낌? 부먹 찍먹 구분없이 그냥 탕...,안녕하세요 너무 감사드립니다 새해에도 건강하시고 행복하세요 감사드립니다 ~,1
23760,면이 조금 불었어 아쉽지만 그래도 맛있게 잘 먹었습니다.,고객님 소중한 리뷰 남겨주셔서 감사합니다. 맛있게 드셨다고 하시니 너무 기분 좋습니...,1


## 형태소 분석을 통한 토큰화

- Rule Based 카테고리 분류
- 입력 -> 맞춤법 검사 -> rule base로 카테고리화 -> 가격 / 맛 / 양 각각에 대해서 몇% 이고, 각각에 대해 긍정인지 부정인지 확인.

In [ ]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=a73386ddf09312d69d3c49a0b544a07e30ab15db72b2bdd0283a980289ecc56f
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [ ]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.8.2-py3-none-any.whl size=39128 sha256=03ded1a2a5d9d9ada3c916be223d7d19daab0ece9e4e7cbd4d0b7aa4766f4f5b
  Stored in directory: /root/.cache/pip/wheels/fd/50/a4/0a8f1bf363cfe5e967da60702ef7c90267e9d00b564ae9b0e5
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=71bda8b333620af098bb1f161fb83ab75825286afde44fd51661313f7e8ab3fd
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


In [ ]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

In [ ]:
kiwi_reviews = []

for review in tqdm(train_ft['review']):
    results = []  # 결과 리스트를 각 리뷰마다 초기화
    result = kiwi.analyze(review)
    for token, pos, _, _ in result[0][0]:
        if pos in ['NNG', 'VA', 'VV', 'XR']:  # 일반명사, 형용사, 동사, 어근
            results.append(token)
    kiwi_reviews.append(results)

  0%|          | 0/23350 [00:00<?, ?it/s]

In [ ]:
train_ft['kiwi_reviews'] = kiwi_reviews
kiwi_reviews

[['김치찌개', '김치', '국', '느낌', '나', '실망', '고기'],
 ['양', '많', '배', '터지', '서비스'],
 ['음식', '부족', '맛', '없', '배달', '늦', '오', '실망'],
 ['잡채', '짜', '하', '맛', '짜', '먹'],
 ['가격', '비하', '음식', '양', '적', '맛', '실망'],
 ['고기', '익', '먹', '없', '서비스', '실망'],
 ['맛', '원', '맛', '서비스'],
 ['밥', '딱딱', '소스', '심심', '맛', '없'],
 ['음식점', '비하', '맛', '떨어지', '양', '적', '실망'],
 ['주문', '음식', '식', '오', '맛', '실망'],
 ['김치찌개', '김치', '국', '느낌', '나', '실망', '고기'],
 ['양', '많', '배', '터지', '서비스'],
 ['맛', '입맛', '맞'],
 ['배달', '늦', '오', '기다리', '피곤'],
 ['맛있', '먹', '기대', '실망', '짜'],
 ['메뉴', '시키', '엉망', '서비스', '맛'],
 ['양', '적', '맛', '시키', '같'],
 ['음식', '맛', '없', '양', '적', '실망'],
 ['배송', '늦', '오', '기다리', '불편'],
 ['맛', '없', '배송', '늦', '오', '서비스'],
 ['김치찌개', '김치', '국', '느낌', '나', '실망', '고기'],
 ['양', '많', '배', '터지', '서비스'],
 ['맛', '기대', '맛있'],
 ['음식', '맛', '없'],
 ['배달', '시간', '걸리', '음식', '따뜻'],
 ['고기', '마르', '먹', '힘들', '맛'],
 ['맛', '느끼', '먹', '양', '많', '남기', '없'],
 ['음식', '짜', '맛', '나', '먹', '힘들'],
 ['맛', '달', '기름지'],
 ['고기', '촉촉', '식', '먹', '그

In [ ]:
train_ft

,review,reply,sentiment,kiwi_reviews
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,"[김치찌개, 김치, 국, 느낌, 나, 실망, 고기]"
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,"[양, 많, 배, 터지, 서비스]"
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,"[음식, 부족, 맛, 없, 배달, 늦, 오, 실망]"
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,"[잡채, 짜, 하, 맛, 짜, 먹]"
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,"[가격, 비하, 음식, 양, 적, 맛, 실망]"
...,...,...,...,...
23757,맛있어요.! 된장찌개에서 멜젓 맛도 나고 맛이있네용 !다만 커피는 제입맛엔 살짝 연...,저희 삼겹살에 찌개까지 맛있게 잘 드셨다니 기쁘고 보람됩니다. 이렇게 소중한 리뷰와...,1,"[맛있, 된장찌개, 멜, 젓, 맛, 나, 맛, 있, 커피, 입맛, 연하, 고기, 맛..."
23758,애들이 이것만 찾아요 바삭하고 짭쪼름하고 맛있어요,맛있게 드셨다니 감사합니다 항상 노력하는 7번가 피자 용암점 되겠습니다 즐거운 시간...,1,"[애, 찾, 짭쪼름, 맛있]"
23759,탕수육 맛있어요 진짜 맛있어요 뭔가 고기가 꽉찬 느낌? 부먹 찍먹 구분없이 그냥 탕...,안녕하세요 너무 감사드립니다 새해에도 건강하시고 행복하세요 감사드립니다 ~,1,"[탕수육, 맛있, 맛있, 고기, 차, 느낌, 부먹, 먹, 구분, 탕수육, 먹, 간,..."
23760,면이 조금 불었어 아쉽지만 그래도 맛있게 잘 먹었습니다.,고객님 소중한 리뷰 남겨주셔서 감사합니다. 맛있게 드셨다고 하시니 너무 기분 좋습니...,1,"[면, 불, 그렇, 맛있, 먹]"


In [ ]:
#200위 내 키워드들 중에 맛, 양, 가격, 서비스, 배달 관련 키워드 추출
taste_keyword = ['맛','맛있','맛나','맛집','매콤하','느끼','신선','냄새','입맛','신선하','건강','매콤','싱싱하','잡내','매운맛','달달하']
quantity_keyword = ['양','푸짐','푸짐하','양도','넉넉','다양','든든','두툼','부족','배부르','든든하'] #'양'추가함
price_keyword = ['가성비','가격','비싸','저렴하']
service_keyword = ['서비스','깔끔','이벤트','친절']
delivery_keyword = ['배달','빠르','배송','도착']

add_taste_keyword = ['맛', '꿀맛', '존맛', '생각나', '추억', '최고', '달', '최악','매콤', '짭짤', '무난', '평범', '먹', '개취', '짱','끝내줘', '괜찮', '푸짐하','단골','단골집', '어울리', '소스', '레전드', '느끼', '식감', '특이', '마싯', '취향', '행복', '양념', '맛있', '대박', '추천', '땡기', '실망', '보통', '독특', '깔끔', '감칠맛', '풍미', '향긋', '입맛', '미각', '조화', '신세계', '맛탱']
add_quantity_keyword = ['혜자','창렬','배 부르','배터지','배터질','충분','풍성','만족스러운 양','양도 많','양이 적','알차','구성','양이 많']
add_price_keyword = ['경제적','저렴','가격 대비','합리적','싸요','비싸요','비싼']
add_service_keyword = ['써비스', '이물질','머리카락','추천','인정','실망','다음','감사','사장','피드백','요청','사항','이벤트','축하','생일','센스','정확','친절','응대','포장']
add_delivery_keyword = ['라이더','신속','안전','빨리', '빠른 배달','도착','예정','시간','여유','기다리','지나도','상태','뜨끈하','기사','한참','배달료','느리','늦','제시간','예상 시간','날씨' ]

taste_keyword = taste_keyword + add_taste_keyword
quantity_keyword = quantity_keyword + add_quantity_keyword
price_keyword = price_keyword + add_price_keyword
service_keyword = service_keyword + add_service_keyword
delivery_keyword = delivery_keyword + add_delivery_keyword

In [ ]:
train_ft['category'] = None

In [ ]:
def get_category(words):
    found_categories = set()  # 중복을 제외한 카테고리를 저장할 집합

    for kiwi in words:
        if kiwi in taste_keyword:
            found_categories.add('맛')
        elif kiwi in quantity_keyword:
            found_categories.add('양')
        elif kiwi in price_keyword:
            found_categories.add('가격')
        elif kiwi in service_keyword:
            found_categories.add('서비스')
        elif kiwi in delivery_keyword:
            found_categories.add('배달')

    # 집합을 리스트로 변환
    unique_categories = list(found_categories)

    return unique_categories

In [ ]:
train_ft['category'] = train_ft['kiwi_reviews'].apply(lambda words: get_category(words))
train_ft

,review,reply,sentiment,kiwi_reviews,category
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,"[김치찌개, 김치, 국, 느낌, 나, 실망, 고기]",[맛]
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,"[양, 많, 배, 터지, 서비스]","[서비스, 양]"
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,"[음식, 부족, 맛, 없, 배달, 늦, 오, 실망]","[배달, 양, 맛]"
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,"[잡채, 짜, 하, 맛, 짜, 먹]",[맛]
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,"[가격, 비하, 음식, 양, 적, 맛, 실망]","[양, 맛, 가격]"
...,...,...,...,...,...
23757,맛있어요.! 된장찌개에서 멜젓 맛도 나고 맛이있네용 !다만 커피는 제입맛엔 살짝 연...,저희 삼겹살에 찌개까지 맛있게 잘 드셨다니 기쁘고 보람됩니다. 이렇게 소중한 리뷰와...,1,"[맛있, 된장찌개, 멜, 젓, 맛, 나, 맛, 있, 커피, 입맛, 연하, 고기, 맛...",[맛]
23758,애들이 이것만 찾아요 바삭하고 짭쪼름하고 맛있어요,맛있게 드셨다니 감사합니다 항상 노력하는 7번가 피자 용암점 되겠습니다 즐거운 시간...,1,"[애, 찾, 짭쪼름, 맛있]",[맛]
23759,탕수육 맛있어요 진짜 맛있어요 뭔가 고기가 꽉찬 느낌? 부먹 찍먹 구분없이 그냥 탕...,안녕하세요 너무 감사드립니다 새해에도 건강하시고 행복하세요 감사드립니다 ~,1,"[탕수육, 맛있, 맛있, 고기, 차, 느낌, 부먹, 먹, 구분, 탕수육, 먹, 간,...",[맛]
23760,면이 조금 불었어 아쉽지만 그래도 맛있게 잘 먹었습니다.,고객님 소중한 리뷰 남겨주셔서 감사합니다. 맛있게 드셨다고 하시니 너무 기분 좋습니...,1,"[면, 불, 그렇, 맛있, 먹]",[맛]


In [ ]:
train_ft = train_ft[train_ft['category'].apply(lambda x: x != '[]' and len(x) > 0)]
train_ft.shape
train_ft

,review,reply,sentiment,kiwi_reviews,category
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,"[김치찌개, 김치, 국, 느낌, 나, 실망, 고기]",[맛]
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,"[양, 많, 배, 터지, 서비스]","[서비스, 양]"
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,"[음식, 부족, 맛, 없, 배달, 늦, 오, 실망]","[배달, 양, 맛]"
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,"[잡채, 짜, 하, 맛, 짜, 먹]",[맛]
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,"[가격, 비하, 음식, 양, 적, 맛, 실망]","[양, 맛, 가격]"
...,...,...,...,...,...
23757,맛있어요.! 된장찌개에서 멜젓 맛도 나고 맛이있네용 !다만 커피는 제입맛엔 살짝 연...,저희 삼겹살에 찌개까지 맛있게 잘 드셨다니 기쁘고 보람됩니다. 이렇게 소중한 리뷰와...,1,"[맛있, 된장찌개, 멜, 젓, 맛, 나, 맛, 있, 커피, 입맛, 연하, 고기, 맛...",[맛]
23758,애들이 이것만 찾아요 바삭하고 짭쪼름하고 맛있어요,맛있게 드셨다니 감사합니다 항상 노력하는 7번가 피자 용암점 되겠습니다 즐거운 시간...,1,"[애, 찾, 짭쪼름, 맛있]",[맛]
23759,탕수육 맛있어요 진짜 맛있어요 뭔가 고기가 꽉찬 느낌? 부먹 찍먹 구분없이 그냥 탕...,안녕하세요 너무 감사드립니다 새해에도 건강하시고 행복하세요 감사드립니다 ~,1,"[탕수육, 맛있, 맛있, 고기, 차, 느낌, 부먹, 먹, 구분, 탕수육, 먹, 간,...",[맛]
23760,면이 조금 불었어 아쉽지만 그래도 맛있게 잘 먹었습니다.,고객님 소중한 리뷰 남겨주셔서 감사합니다. 맛있게 드셨다고 하시니 너무 기분 좋습니...,1,"[면, 불, 그렇, 맛있, 먹]",[맛]


# 서비스2 모델링 진행

In [ ]:
result_df = train_ft.copy()
result_df

,review,reply,sentiment,score,kiwi_reviews,category
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.566494,"['맛있', '먹', '묵사발', '시원', '맛있']",['맛']
1,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,고객님께서도 행복하시길 바랄게요,1,0.608408,"['양', '푸짐', '맛있', '청국장', '냄새', '나', '콩', '들', ...","['맛', '양']"
2,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,고객님의 행복한 시간에 저희 매장이 함께 할 수 있어 너무나도 영광입니다! 즐거움이...,1,0.381793,"['냉면', '시키', '미니', '냉면', '합치', '양', '적', '보이',...",['양']
3,맛나게 잘 묵었습니다,"고객님의 즐거운 순간을 함께할 수 있기를 바라며, 기다리고 있겠습니다 더욱 발전된 ...",1,0.549870,"['맛나', '묵']",['맛']
4,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",1,0.778149,"['맛', '양', '만족', '배달', '보내', '만족']","['맛', '양', '배달']"
...,...,...,...,...,...,...
42539,언제나 맛있게 먹었는데 이번에는 음식이 지루하고 별로였어요. 아쉬워요,"닭도리탕의 맛이 매우 좋지 않고, 김치가 물렸다는 점 죄송합니다. 앞으로 더 맛있는...",0,0.827452,"['맛있', '먹', '이번', '음식', '지루하']",['맛']
42540,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...",고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하셨다니 정말 ...,0,0.942769,"['맛', '음식', '촉촉', '식', '먹', '그렇', '다음', '시키', ...","['맛', '서비스']"
42541,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723,"['맛', '없', '배달', '늦', '실망']","['맛', '배달']"
42542,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588,"['최악', '맛', '없', '배달', '걸리']","['맛', '배달']"


In [ ]:
result_df["category"].unique()

array(["['맛']", "['맛', '양']", "['양']", "['맛', '양', '배달']", "['맛', '서비스']",
       "['맛', '배달']", "['맛', '양', '가격']", "['배달']",
       "['맛', '서비스', '양', '배달']", "['맛', '서비스', '배달']",
       "['맛', '서비스', '양']", "['맛', '가격']", "['맛', '서비스', '가격']",
       "['서비스', '배달']", "['배달', '맛', '양', '가격', '서비스']",
       "['맛', '서비스', '양', '가격']", "['맛', '배달', '가격']",
       "['맛', '양', '배달', '가격']", "['서비스', '양', '배달']", "['서비스', '양']",
       "['양', '배달']", "['양', '배달', '가격']", "['맛', '서비스', '배달', '가격']",
       "['양', '가격']", "['배달', '가격']", "['서비스', '양', '배달', '가격']",
       "['서비스', '양', '가격']", "['서비스', '배달', '가격']"], dtype=object)

In [ ]:
result_df.rename(columns={
    'review': 'Review',
    'category': 'Label'
}, inplace=True)
result_df = result_df[["Review", "Label"]]

result_df = result_df.dropna().reset_index(drop=True)

In [ ]:
result_df

,Review,Label
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,['맛']
1,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,"['맛', '양']"
2,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,['양']
3,맛나게 잘 묵었습니다,['맛']
4,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"['맛', '양', '배달']"
...,...,...
42539,언제나 맛있게 먹었는데 이번에는 음식이 지루하고 별로였어요. 아쉬워요,['맛']
42540,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","['맛', '서비스']"
42541,"맛도 없고, 배달도 늦어서 너무 실망했습니다","['맛', '배달']"
42542,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,"['맛', '배달']"


- 학습, 검증데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(result_df, test_size=0.05, random_state=SEED)

In [ ]:
train_df

,Review,Label
24139,맛있게 잘 먹었습니다,['맛']
40083,이번에 시킨 음식은 좀 싱겁고 맛이 없어서 아쉬웠어요. 다른 음식들은 맛있는데 이게...,['맛']
8869,맛잇네요. 다음엔 매운맛으로 도전해야겠어요,"['맛', '서비스']"
26245,아이들만 시켜주었는데 집에 오니 그릇만 돌아다니네요 맛있었다고 합니다,['맛']
22225,맛있게 먹었습니다 고추까지 추가해서 넣어먹으니까 더 맛있는거 같아요. 양도 푸짐하고...,"['맛', '양']"
...,...,...
6265,이 메뉴가 다양하게 맛볼수 있고 가성비 갑인것 같아요 망설이지 마시고 드세요!,"['양', '가격']"
11284,처음 시켜봤는데 배달도빠르구 진짜맛잇엇어요 특히 서비스로주신 초밥 이거몬가요 너무맛...,"['맛', '서비스', '배달']"
38158,음식이 너무 짜서 한 입 먹으면 물 마시러 가야 했어요. 좀 덜 짜면 좋을 것 같아요,['맛']
860,배달기사분이 도착 십분전에 미리 문앞에 나와있게해서 얼어죽는줄 알았어요 이런경우 처...,"['맛', '서비스', '배달']"


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
type(train_df["Label"])

pandas.core.series.Series

In [ ]:
import pandas as pd
import numpy as np

# 레이블을 숫자로 변환하는 딕셔너리
candidate_labels  = {0: '맛', 1: '양', 2: '가격', 3: '배달', 4: '서비스'}

def string_to_list(label_str):
    if isinstance(label_str, str):
        # 작은 따옴표를 제거하고 split
        return [label.replace("'", "").strip() for label in label_str[1:-1].split(',')]
    return label_str

# labels_to_onehot 함수 수정
def labels_to_onehot(labels, candidate_labels ):
    onehot = [0]*len(candidate_labels )
    for label in labels:
        for key, value in candidate_labels .items():
            if label == value:
                onehot[key] = 1
    return onehot

# train_df 데이터에 대한 문자열 레이블을 리스트로 변환
train_df['Label_List'] = train_df['Label'].apply(string_to_list)

# train_df 데이터에 대한 리스트 형태의 레이블을 one-hot encoding으로 변환
train_df['OneHot_Labels'] = train_df['Label_List'].apply(lambda x: labels_to_onehot(x, candidate_labels ))

# valid_df 데이터에 대한 문자열 레이블을 리스트로 변환
valid_df['Label_List'] = valid_df['Label'].apply(string_to_list)

# valid_df 데이터에 대한 리스트 형태의 레이블을 one-hot encoding으로 변환
valid_df['OneHot_Labels'] = valid_df['Label_List'].apply(lambda x: labels_to_onehot(x, candidate_labels ))

# train_df 데이터 프레임의 일부를 출력하여 확인
print(train_df[['Review', 'Label_List', 'OneHot_Labels']].head())

                                                  Review Label_List  \
24139                                       맛있게 잘 먹었습니다         [맛]   
40083  이번에 시킨 음식은 좀 싱겁고 맛이 없어서 아쉬웠어요. 다른 음식들은 맛있는데 이게...        [맛]   
8869                             맛잇네요. 다음엔 매운맛으로 도전해야겠어요   [맛, 서비스]   
26245             아이들만 시켜주었는데 집에 오니 그릇만 돌아다니네요 맛있었다고 합니다        [맛]   
22225  맛있게 먹었습니다 고추까지 추가해서 넣어먹으니까 더 맛있는거 같아요. 양도 푸짐하고...     [맛, 양]   

         OneHot_Labels  
24139  [1, 0, 0, 0, 0]  
40083  [1, 0, 0, 0, 0]  
8869   [1, 0, 0, 0, 1]  
26245  [1, 0, 0, 0, 0]  
22225  [1, 1, 0, 0, 0]  


In [ ]:
train_df[['Review', 'Label_List', 'OneHot_Labels']]

,Review,Label_List,OneHot_Labels
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,[맛],"[1, 0, 0, 0, 0]"
1,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,"[맛, 양]","[1, 1, 0, 0, 0]"
2,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,[양],"[0, 1, 0, 0, 0]"
3,맛나게 잘 묵었습니다,[맛],"[1, 0, 0, 0, 0]"
4,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"[맛, 양, 배달]","[1, 1, 0, 1, 0]"
...,...,...,...
39995,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","[맛, 서비스]","[1, 0, 0, 0, 1]"
39996,배달이 너무 느려서 식사 시간이 피해버렸어요. 이 정도면 좀 불편했어요,[배달],"[0, 0, 0, 1, 0]"
39997,맛도 없고 양도 적었어요. 돈 아깝게 주고 먹은 것 같아요,"[맛, 양]","[1, 1, 0, 0, 0]"
39998,음식이 싱거워서 먹을 맛이 없었어요. 다른 식당을 찾아봐야겠어요,[맛],"[1, 0, 0, 0, 0]"


In [ ]:
valid_df[['Review', 'Label_List', 'OneHot_Labels']]

,Review,Label_List,OneHot_Labels
40000,배달기사 분이 무례하게 대응해서 기분이 상했어요. 다시 시키지 않을 거에요,[배달],"[0, 0, 0, 1, 0]"
40001,음식이 변색되어 있었고 맛도 이상했어요. 좀 더 신선한 음식을 원했는데 아쉬웠어요,[맛],"[1, 0, 0, 0, 0]"
40002,음식이 늦게 도착했고 따뜻하지 않아서 맛도 별로였어요. 좀 더 신경써서 준비해주세요,"[맛, 배달]","[1, 0, 0, 1, 0]"
40003,주문한 음식이 다른 음식으로 바뀌어서 실망했어요. 이런 일이 또 발생하면 안될 것 ...,[맛],"[1, 0, 0, 0, 0]"
40004,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","[맛, 서비스]","[1, 0, 0, 0, 1]"
...,...,...,...
42495,음식이 너무 많이 짜서 입맛에 잘 맞지 않았어요. 별로였어요,[맛],"[1, 0, 0, 0, 0]"
42496,음식이 너무 느끼하고 육즙이 없어서 싱겁게 먹기가 어려웠어요. 별로였어요,[맛],"[1, 0, 0, 0, 0]"
42497,음식이 너무 뜨거워서 먹기가 힘들었어요. 맛도 별로였어요,[맛],"[1, 0, 0, 0, 0]"
42498,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","[맛, 서비스]","[1, 0, 0, 0, 1]"


## 사전학습모델 불러오기

In [ ]:
import numpy as np
from transformers import AutoTokenizer

# 모델 이름 및 토크나이저 정의
model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# 주어진 데이터프레임에서 텍스트와 라벨을 인코딩하는 함수
def process_data(df, tokenizer):
    texts = df["Review"].tolist()
    labels_encoded = np.array(df["OneHot_Labels"].tolist())

    # 텍스트 인코딩
    encodings = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")
    return encodings, labels_encoded

# 각 데이터셋에 대한 처리
train_encodings, train_labels_encoded = process_data(train_df, tokenizer)
val_encodings, val_labels_encoded = process_data(valid_df, tokenizer)

In [ ]:
# 첫 5개의 훈련 데이터 및 라벨 출력
for i in range(5):
    print("Review:", train_df["Review"].iloc[i])
    print("Encoded:", train_encodings['input_ids'][i])
    print("Label:", train_labels_encoded[i])
    print("-" * 50)

# 첫 5개의 검증 데이터 및 라벨 출력
for i in range(5):
    print("Review:", valid_df["Review"].iloc[i])
    print("Encoded:", val_encodings['input_ids'][i])
    print("Label:", val_labels_encoded[i])
    print("-" * 50)

Review: 맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요
Encoded: tensor([     1,    260,  35099,  54198,   2318,  14985,  82509,  76883,    260,
        126646,   3073,   9381,   1248,   6464,   3348, 138603,    260,  35099,
         54198,  38646,      2,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      

## 데이터셋 만들기

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float).to('cuda')  # dtype을 명시적으로 설정
        return item

    def __len__(self):
        return len(self.labels)

# 데이터를 데이터셋으로 변환
train_dataset = CustomDataset(train_encodings, train_labels_encoded)
val_dataset = CustomDataset(val_encodings, val_labels_encoded)

## 데이터로더 만들기

In [ ]:
from torch.utils.data import DataLoader

# DataLoader 설정
batch_size = 16  # 이 값은 메모리 및 학습 설정에 따라 조정해야 할 수 있습니다.

train_dataset = CustomDataset(train_encodings, train_labels_encoded)
val_dataset = CustomDataset(val_encodings, val_labels_encoded)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW
from torch.nn import BCEWithLogitsLoss

# 모델, 손실 함수, 옵티마이저 초기화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, ignore_mismatched_sizes=True).to('cuda')
loss_fn = BCEWithLogitsLoss().to('cuda')
optimizer = AdamW(model.parameters(), lr=1e-5)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at MoritzLaurer/mDeBERTa-v3-base-mnli-xnli and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 학습하기

In [ ]:
from tqdm import tqdm
import time

epochs = 5
total_steps = len(train_loader) * epochs
model.train()

for epoch in range(epochs):
    total_loss = 0
    epoch_start_time = time.time()  # 에폭 시작 시간 저장

    # tqdm을 사용하여 학습의 진행 상황을 보여줌
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):

        # 배치 데이터를 CUDA에 할당
        inputs = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')

        # 예측 및 손실 계산
        outputs = model(inputs, attention_mask=attention_mask)
        logits = outputs.logits
        loss = loss_fn(logits, labels.float())

        # 역전파
        loss.backward()

        # 가중치 업데이트
        optimizer.step()

        # 그래디언트 초기화
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    epoch_time_elapsed = time.time() - epoch_start_time  # 에폭 경과 시간 계산

    # 학습 로스와 에폭 경과 시간 출력
    print(f"  평균 학습 loss: {avg_train_loss:.4f}")
    print(f"  에폭 토크 학습: {epoch_time_elapsed:.2f}s")

    model.save_pretrained(f"{DATA_PATH}42000_5라벨_{epoch+1}")
    tokenizer.save_pretrained(f"{DATA_PATH}42000_5라벨_{epoch+1}")
    print(f"  에폭 저장 {epoch+1}")

print("완료")


Epoch 1/5: 100%|██████████| 2500/2500 [38:57<00:00,  1.07it/s]


  평균 학습 loss: 0.0898
  에폭 토크 학습: 2337.30s
  에폭 저장 1


Epoch 2/5: 100%|██████████| 2500/2500 [38:57<00:00,  1.07it/s]


  평균 학습 loss: 0.0236
  에폭 토크 학습: 2337.08s
  에폭 저장 2


Epoch 3/5: 100%|██████████| 2500/2500 [38:56<00:00,  1.07it/s]


  평균 학습 loss: 0.0164
  에폭 토크 학습: 2336.63s
  에폭 저장 3


Epoch 4/5: 100%|██████████| 2500/2500 [38:56<00:00,  1.07it/s]


  평균 학습 loss: 0.0136
  에폭 토크 학습: 2336.91s
  에폭 저장 4


Epoch 5/5: 100%|██████████| 2500/2500 [38:58<00:00,  1.07it/s]


  평균 학습 loss: 0.0111
  에폭 토크 학습: 2338.22s
  에폭 저장 5
완료


In [ ]:
model_zeroshot = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/프로젝트/final project/data/42000_5라벨_5").to('cuda')

In [ ]:
print(model)
print(model.config)

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(251000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

## 검증하기

In [ ]:
model_zeroshot = model_zeroshot.to('cuda')

model_zeroshot.eval()  # 모델을 평가 모드로 설정

total_loss = 0
correct_predictions = 0
total_predictions = 0

for batch in val_loader:
    with torch.no_grad():  # 기울기 계산을 하지 않음
        inputs = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')

        outputs = model_zeroshot(inputs, attention_mask=attention_mask)
        logits = outputs.logits
        loss = loss_fn(logits, labels.float())
        total_loss += loss.item()

        # 로지스틱 회귀 결과를 확률로 변환
        probs = torch.sigmoid(logits)
        # 확률을 라벨 (0 or 1)로 변환
        preds = (probs > 0.5).long()

        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.numel()

avg_val_loss = total_loss / len(val_loader)
accuracy = correct_predictions / total_predictions

print(f"Validation Loss: {avg_val_loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


Validation Loss: 0.0060
Validation Accuracy: 0.9987


## 추론하기

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# 데이터 추출
valid_df = valid_df["Review"].tolist()

# 1. 데이터 전처리
inputs = tokenizer(valid_df, truncation=True, padding=True, return_tensors="pt")

# TensorDataset 및 DataLoader 구성
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
data_loader = DataLoader(dataset, batch_size=16, shuffle=False)  # 배치 크기는 메모리에 따라 조정할 수 있습니다.

# 2. 모델 예측
all_preds = []
with torch.no_grad():
    for batch in data_loader:
        input_ids, attention_mask = (item.to('cuda') for item in batch)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).long()  # 확률을 라벨 (0 or 1)로 변환
        all_preds.extend(preds.cpu().tolist())

print(all_preds)

[[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 0, 1], [0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 

- 멀티 라벨링으로 변환

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset

# 레이블을 숫자로 변환하는 딕셔너리
label_to_num = {0: '맛', 1: '양', 2: '가격', 3: '배달', 4: '서비스'}

# 모델 및 토크나이저 불러오기
# model = AutoModelForSequenceClassification.from_pretrained("rere_30000").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", use_fast=False)


# 데이터 추출
valid_df = valid_df["Review"].tolist()

# 1. 데이터 전처리
inputs = tokenizer(valid_df, truncation=True, padding=True, return_tensors="pt")

# TensorDataset 및 DataLoader 구성
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
data_loader = DataLoader(dataset, batch_size=16, shuffle=False)

# 2. 모델 예측
all_preds = []
with torch.no_grad():
    for batch in data_loader:
        input_ids, attention_mask = (item.to('cuda') for item in batch)
        outputs = model_zeroshot(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).long()  # 확률을 라벨 (0 or 1)로 변환
        all_preds.extend(preds.cpu().tolist())

# 예측된 원-핫 인코딩된 라벨을 텍스트 라벨로 변환하는 함수
def one_hot_to_labels(one_hot_labels, label_to_num):
    labels_list = []
    for one_hot in one_hot_labels:
        labels = [label_to_num[i] for i, value in enumerate(one_hot) if value == 1]
        labels_list.append(labels)
    return labels_list

# 예측된 원-핫 인코딩된 라벨을 텍스트 라벨로 변환
predicted_label_list = one_hot_to_labels(all_preds, label_to_num)

# 예측된 라벨의 시작과 끝 인덱스 계산
start_index = len(result_df) - len(predicted_label_list)
end_index = start_index + len(predicted_label_list)

# 예측된 라벨을 추가
result_df["Predicted_Labels"] = [""] * start_index + predicted_label_list + [""] * (len(result_df) - end_index)

# 결과 출력
print(result_df[['Review', 'Predicted_Labels']].iloc[start_index:end_index])
result_df.to_csv(DATA_PATH + "dynamic_zeroshot_fine.csv", index=False)

                                                  Review Predicted_Labels
40000         배달기사 분이 무례하게 대응해서 기분이 상했어요. 다시 시키지 않을 거에요           [맛, 배달]
40001     음식이 변색되어 있었고 맛도 이상했어요. 좀 더 신선한 음식을 원했는데 아쉬웠어요               [맛]
40002    음식이 늦게 도착했고 따뜻하지 않아서 맛도 별로였어요. 좀 더 신경써서 준비해주세요           [맛, 배달]
40003  주문한 음식이 다른 음식으로 바뀌어서 실망했어요. 이런 일이 또 발생하면 안될 것 ...              [맛]
40004  맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...         [맛, 서비스]
...                                                  ...              ...
41995                              맛도 별로고 배달이 느려서 아쉬웠어요           [맛, 배달]
41996                             배달이 너무 느리고 음식도 맛이 없어요           [맛, 배달]
41997                    배달 시간이 오래 걸리고 음식도 맛이 없어서 실망했어요           [맛, 배달]
41998                   음식이 맛이 없었어요. 다른 가게에서 시킬 걸 그랬어요.               [맛]
41999                  음식이 식어서 맛이 없어졌어요. 다신 시키지 않을 거예요.               [맛]

[2000 rows x 2 columns]


In [ ]:
result_df[40000:42000]

,Review,Label,Predicted_Labels
40000,배달기사 분이 무례하게 대응해서 기분이 상했어요. 다시 시키지 않을 거에요,['배달'],"[맛, 배달]"
40001,음식이 변색되어 있었고 맛도 이상했어요. 좀 더 신선한 음식을 원했는데 아쉬웠어요,['맛'],[맛]
40002,음식이 늦게 도착했고 따뜻하지 않아서 맛도 별로였어요. 좀 더 신경써서 준비해주세요,"['맛', '배달']","[맛, 배달]"
40003,주문한 음식이 다른 음식으로 바뀌어서 실망했어요. 이런 일이 또 발생하면 안될 것 ...,['맛'],[맛]
40004,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","['맛', '서비스']","[맛, 서비스]"
...,...,...,...
41995,맛도 별로고 배달이 느려서 아쉬웠어요,"['맛', '배달']","[맛, 배달]"
41996,배달이 너무 느리고 음식도 맛이 없어요,"['맛', '배달']","[맛, 배달]"
41997,배달 시간이 오래 걸리고 음식도 맛이 없어서 실망했어요,"['맛', '배달']","[맛, 배달]"
41998,음식이 맛이 없었어요. 다른 가게에서 시킬 걸 그랬어요.,['맛'],[맛]


In [ ]:
reviews_to_predict = [
    '맛은 있는데 양은 적어요',
    '가성비가 별로 안좋아요',
    '배 터질것 같아요',
    '이렇게 늦을 거면 안 시켜 먹었죠',
    '가격에 비해 양이 좀 적어요',
    '맛있는데 너무 늦게 왔어요',
    '맛에 비해 너무 비싸요',
    '다시는 안 시킬 것 같아요',
    '또 시켜먹을게요',
    '주문하고 30분 내로 왔어요',
    '혜자스럽네요',
    '창렬스럽네요',
    '이물질이 나왔어요',
    '국밥 국물이 끝내줘요',
    '치킨은 매번 여기서 시켜먹어요',
]

# Fine-tuned 모델 + Base 모델 앙상블
- task 특성상 Fine-tuned 모델만 사용시 과적합이 발생
- Fine-tuning 하지 않은 기존 base 모델(MoritzLaurer/mDeBERTa-v3-base-mnli-xnli)과 앙상블하여 모델 일반화 진행

## Fine-tuned 모델

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# model_zeroshot
candidate_labels = ['맛', '양', '가격', '배달', '서비스']

# 모델 및 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", use_fast=False)

# 데이터 전처리
inputs = tokenizer(reviews_to_predict, truncation=True, padding=True, return_tensors="pt").to('cuda')

# DataLoader 구성
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
data_loader = DataLoader(dataset, batch_size=16, shuffle=False)

labels_collected = []
scores_collected = []

with torch.no_grad():
    for batch in tqdm(data_loader):
        input_ids, attention_mask = (item.to('cuda') for item in batch)
        outputs = model_zeroshot(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()

        for prob in probs:
            labels = [candidate_labels[idx] for idx, val in enumerate(prob) if val > 0.5]
            scores = {candidate_labels[idx]: f"{val:.2f}" for idx, val in enumerate(prob)}
            labels_collected.append(", ".join(labels))
            scores_collected.append(", ".join([f"{k}: {v}" for k, v in scores.items()]))

# 결과를 데이터 프레임으로 저장
result_df = pd.DataFrame({
    'Review': reviews_to_predict,
    'Predicted_Labels': labels_collected,
    'Label_Scores': scores_collected
})

result_df

100%|██████████| 1/1 [00:00<00:00, 14.34it/s]


,Review,Predicted_Labels,Label_Scores
0,맛은 있는데 양은 적어요,"맛, 양","맛: 1.00, 양: 0.99, 가격: 0.00, 배달: 0.00, 서비스: 0.00"
1,가성비가 별로 안좋아요,"맛, 가격","맛: 1.00, 양: 0.00, 가격: 1.00, 배달: 0.00, 서비스: 0.01"
2,배 터질것 같아요,맛,"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"
3,이렇게 늦을 거면 안 시켜 먹었죠,맛,"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"
4,가격에 비해 양이 좀 적어요,"양, 가격","맛: 0.25, 양: 0.99, 가격: 1.00, 배달: 0.00, 서비스: 0.01"
5,맛있는데 너무 늦게 왔어요,맛,"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"
6,맛에 비해 너무 비싸요,"맛, 가격","맛: 1.00, 양: 0.00, 가격: 1.00, 배달: 0.00, 서비스: 0.01"
7,다시는 안 시킬 것 같아요,맛,"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"
8,또 시켜먹을게요,맛,"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"
9,주문하고 30분 내로 왔어요,맛,"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"


## Base 모델
- MoritzLaurer/mDeBERTa-v3-base-mnli-xnli
    - Fine-tuning 하지 않은 기본 사전학습모델

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm

# 모델과 토크나이저 초기화
model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 제로샷 분류 파이프라인 초기화
classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer, device=0)  # GPU 사용 설정

# 분류를 위한 레이블 목록
candidate_labels = ['맛', '양', '가격', '배달', '서비스']

# 전체 리뷰에 대해 제로샷 분류 수행
labels_collected = []
scores_collected = []

for review in tqdm(reviews_to_predict):  # 리스트를 바로 순회
    output = classifier(review, candidate_labels, multi_label=True)
    labels = [label for label, score in zip(output['labels'], output['scores']) if score > 0.5]
    scores = [round(score, 2) for score in output['scores']]

    labels_collected.append(', '.join(labels))
    scores_collected.append(', '.join([f"{label}: {round(score, 2)}" for label, score in zip(output['labels'], output['scores'])]))

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame({
    'Review': reviews_to_predict,
    'Predicted_Labels': labels_collected,
    'Label_Scores': scores_collected
})

result_df

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
 67%|██████▋   | 10/15 [00:02<00:00,  5.21it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 15/15 [00:03<00:00,  4.92it/s]


,Review,Predicted_Labels,Label_Scores
0,맛은 있는데 양은 적어요,맛,"맛: 0.98, 가격: 0.34, 서비스: 0.33, 배달: 0.25, 양: 0.12"
1,가성비가 별로 안좋아요,가격,"가격: 0.93, 배달: 0.42, 양: 0.35, 서비스: 0.23, 맛: 0.01"
2,배 터질것 같아요,"양, 배달, 가격","양: 0.88, 배달: 0.59, 가격: 0.59, 서비스: 0.1, 맛: 0.08"
3,이렇게 늦을 거면 안 시켜 먹었죠,"서비스, 맛, 가격, 배달","서비스: 0.95, 맛: 0.75, 가격: 0.64, 배달: 0.52, 양: 0.25"
4,가격에 비해 양이 좀 적어요,"가격, 양","가격: 0.81, 양: 0.63, 서비스: 0.45, 배달: 0.33, 맛: 0.01"
5,맛있는데 너무 늦게 왔어요,"맛, 배달, 서비스","맛: 1.0, 배달: 0.98, 서비스: 0.88, 양: 0.39, 가격: 0.27"
6,맛에 비해 너무 비싸요,가격,"가격: 1.0, 배달: 0.19, 맛: 0.13, 양: 0.1, 서비스: 0.08"
7,다시는 안 시킬 것 같아요,서비스,"서비스: 0.64, 맛: 0.48, 가격: 0.48, 양: 0.46, 배달: 0.23"
8,또 시켜먹을게요,"서비스, 맛, 가격","서비스: 0.99, 맛: 0.94, 가격: 0.57, 양: 0.22, 배달: 0.12"
9,주문하고 30분 내로 왔어요,"서비스, 배달, 맛","서비스: 0.97, 배달: 0.75, 맛: 0.69, 가격: 0.41, 양: 0.27"


## 산술평균 예측
- 평균 0.5 이상인 토픽만 반환

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm import tqdm
import pandas as pd

# 첫 번째 모델
# zeroshot_model30000 = AutoModelForSequenceClassification.from_pretrained("/path/to/model").to('cuda')

candidate_labels = ['맛', '양', '가격', '배달', '서비스']

# 두 번째 모델
model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer, device=0)

# 데이터 전처리
inputs = tokenizer(reviews_to_predict, truncation=True, padding=True, return_tensors="pt").to('cuda')
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
data_loader = DataLoader(dataset, batch_size=16, shuffle=False)

# 모델 1 예측
scores_from_model1 = []

with torch.no_grad():
    for batch in tqdm(data_loader):
        input_ids, attention_mask = (item.to('cuda') for item in batch)
        outputs = model_zeroshot(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()

        for prob in probs:
            scores = {candidate_labels[idx]: f"{val:.2f}" for idx, val in enumerate(prob)}
            scores_from_model1.append(scores)

# 모델 2 예측
scores_from_model2 = []

for review in tqdm(reviews_to_predict):
    output = classifier(review, candidate_labels, multi_label=True)
    scores = {label: round(score, 2) for label, score in zip(output['labels'], output['scores'])}
    scores_from_model2.append(scores)

new_label_scores_list = []
new_predicted_labels_list = []

for score1, score2 in zip(scores_from_model1, scores_from_model2):
    averaged_scores = {}
    new_labels = []
    new_scores = {}
    service_scores = []

    for label in score1.keys():
        averaged_score = (float(score1[label]) + float(score2[label])) / 2

        if label == '맛':
            new_scores['quality'] = averaged_score
            if averaged_score >= 0.35:
                new_labels.append('quality')

        elif label == '양':
            new_scores['quantity'] = averaged_score
            if averaged_score >= 0.35:
                new_labels.append('quantity')

        elif label in ['서비스', '배달', '가격']:
            service_scores.append(averaged_score)

    if service_scores:
        avg_service_score = sum(service_scores) / len(service_scores)
        new_scores['service'] = avg_service_score
        if avg_service_score >= 0.35:
            new_labels.append('service')

    new_label_scores_list.append(", ".join([f"{k}: {v:.2f}" for k, v in new_scores.items()]))
    new_predicted_labels_list.append(", ".join(new_labels))

# 결과를 새로운 데이터프레임으로 저장
new_result_df = pd.DataFrame({
    'Review': reviews_to_predict,
    'Label_Scores': new_label_scores_list,
    'New_Predicted_Labels': new_predicted_labels_list
})

new_result_df

 67%|██████▋   | 10/15 [00:01<00:00,  7.83it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 15/15 [00:02<00:00,  6.97it/s]


,Review,Label_Scores,New_Predicted_Labels
0,맛은 있는데 양은 적어요,"quality: 0.99, quantity: 0.55, service: 0.15","quality, quantity"
1,가성비가 별로 안좋아요,"quality: 0.51, quantity: 0.17, service: 0.43","quality, service"
2,배 터질것 같아요,"quality: 0.54, quantity: 0.44, service: 0.21","quality, quantity"
3,이렇게 늦을 거면 안 시켜 먹었죠,"quality: 0.88, quantity: 0.12, service: 0.35","quality, service"
4,가격에 비해 양이 좀 적어요,"quality: 0.13, quantity: 0.81, service: 0.43","quantity, service"
5,맛있는데 너무 늦게 왔어요,"quality: 1.00, quantity: 0.20, service: 0.35","quality, service"
6,맛에 비해 너무 비싸요,"quality: 0.56, quantity: 0.05, service: 0.38","quality, service"
7,다시는 안 시킬 것 같아요,"quality: 0.74, quantity: 0.23, service: 0.23",quality
8,또 시켜먹을게요,"quality: 0.97, quantity: 0.11, service: 0.28",quality
9,주문하고 30분 내로 왔어요,"quality: 0.84, quantity: 0.14, service: 0.35","quality, service"


## 검증하기

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset

# 레이블을 숫자로 변환하는 딕셔너리
label_to_num = {0: '맛', 1: '양', 2: '가격', 3: '배달', 4: '서비스'}

# 모델 및 토크나이저 불러오기
# model = AutoModelForSequenceClassification.from_pretrained("rere_30000").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", use_fast=False)

# 데이터 추출
valid_df = valid_df["Review"].tolist()

# 1. 데이터 전처리
inputs = tokenizer(valid_df, truncation=True, padding=True, return_tensors="pt")

# TensorDataset 및 DataLoader 구성
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
data_loader = DataLoader(dataset, batch_size=16, shuffle=False)

# 2. 모델 예측
all_preds = []
all_probs = []  # 확률값을 저장할 리스트
with torch.no_grad():
    for batch in data_loader:
        input_ids, attention_mask = (item.to('cuda') for item in batch)
        outputs = model_zeroshot(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).long()
        all_preds.extend(preds.cpu().tolist())
        all_probs.extend(probs.cpu().tolist())  # 확률값 저장


# 예측된 원-핫 인코딩된 라벨을 텍스트 라벨로 변환하는 함수
def one_hot_to_labels(one_hot_labels, label_to_num):
    labels_list = []
    for one_hot in one_hot_labels:
        labels = [label_to_num[i] for i, value in enumerate(one_hot) if value == 1]
        labels_list.append(labels)
    return labels_list

# 예측된 원-핫 인코딩된 라벨을 텍스트 라벨로 변환
predicted_label_list = one_hot_to_labels(all_preds, label_to_num)

# 확률값을 문자열로 변환하는 함수
def probs_to_str(probs, label_to_num):
    return ", ".join([f"{label_to_num[i]}: {prob:.2f}" for i, prob in enumerate(probs)])

# 확률값을 문자열로 변환
label_scores_list = [probs_to_str(probs, label_to_num) for probs in all_probs]


# 예측된 라벨과 확률값의 시작과 끝 인덱스 계산
start_index = len(result_df) - len(predicted_label_list)
end_index = start_index + len(predicted_label_list)

# 예측된 라벨과 확률값을 추가
result_df["Predicted_Labels"] = [""] * start_index + predicted_label_list + [""] * (len(result_df) - end_index)
result_df["Label_Scores"] = [""] * start_index + label_scores_list + [""] * (len(result_df) - end_index)

# 결과 출력
print(result_df[['Review', 'Predicted_Labels', 'Label_Scores']].iloc[start_index:end_index])

,Review,Predicted_Labels,Label_Scores
40000,배달기사 분이 무례하게 대응해서 기분이 상했어요. 다시 시키지 않을 거에요,"[맛, 배달]","맛: 0.89, 양: 0.00, 가격: 0.00, 배달: 1.00, 서비스: 0.14"
40001,음식이 변색되어 있었고 맛도 이상했어요. 좀 더 신선한 음식을 원했는데 아쉬웠어요,[맛],"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"
40002,음식이 늦게 도착했고 따뜻하지 않아서 맛도 별로였어요. 좀 더 신경써서 준비해주세요,"[맛, 배달]","맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 1.00, 서비스: 0.01"
40003,주문한 음식이 다른 음식으로 바뀌어서 실망했어요. 이런 일이 또 발생하면 안될 것 ...,[맛],"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"
40004,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","[맛, 서비스]","맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 1.00"
...,...,...,...
41995,맛도 별로고 배달이 느려서 아쉬웠어요,"[맛, 배달]","맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 1.00, 서비스: 0.01"
41996,배달이 너무 느리고 음식도 맛이 없어요,"[맛, 배달]","맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 1.00, 서비스: 0.01"
41997,배달 시간이 오래 걸리고 음식도 맛이 없어서 실망했어요,"[맛, 배달]","맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 1.00, 서비스: 0.00"
41998,음식이 맛이 없었어요. 다른 가게에서 시킬 걸 그랬어요.,[맛],"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"


- 토픽별 평균 확률값

In [ ]:
from collections import defaultdict

# 모든 확률값에 대한 평균을 구하는 함수
def calculate_average_scores(all_probs, label_to_num):
    # 레이블 별 확률값의 합계와 개수를 저장할 딕셔너리
    score_sums = defaultdict(float)
    score_counts = defaultdict(int)
    print(f"전체 샘플 수: {len(all_probs)}")
    # 모든 확률값을 순회하면서 합계와 개수를 계산
    for probs in all_probs:
        for i, prob in enumerate(probs):
            label = label_to_num[i]
            score_sums[label] += prob
            score_counts[label] += 1

    # 레이블 별 평균 확률값을 계산
    average_scores = {label: score_sums[label] / score_counts[label] for label in label_to_num.values()}

    return average_scores

# 모든 확률값에 대한 평균을 계산 (all_probs에 실제 확률값 리스트를 사용해야 함)
average_scores = calculate_average_scores(all_probs, label_to_num)

# 평균 확률값 출력
print("평균 확률값:")
for label, average_score in average_scores.items():
    print(f"{label}: {average_score:.2f}")

전체 샘플 수: 2000
평균 확률값:
맛: 0.93
양: 0.10
가격: 0.06
배달: 0.19
서비스: 0.21
